In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../scripts/")

In [4]:
from msmarco_dataset import MsMarcoDataset
from args_parser import getArgs

In [8]:
argv = ["--data_dir", "/ssd2/arthur/TREC2019/data", 
        "--train_file", "/ssd2/arthur/insy/msmarco/data/train-triples.1",
        "--dev_file", "/ssd2/arthur/insy/msmarco/data/dev-triples.1",
        "--bert-model", "bert-base-uncased"]
args = getArgs(argv)

In [9]:
train_dataset = MsMarcoDataset(args.train_file, args.data_dir)
dev_dataset  = MsMarcoDataset(args.dev_file, args.data_dir)

In [10]:
from sentence_level_classifier import fine_tune

In [64]:
from pytorch_transformers import AdamW, WarmupLinearSchedule
def init_optimizer(
        model: BertForNextSentencePrediction,
        n_steps, lr,
        warmup_proportion=0.1,
        weight_decay=0.0):

    no_decay = ['bias', 'LayerNorm.weight']

    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(
            nd in n for nd in no_decay)], 'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(
            nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=lr, eps=1e-8)
    warmup_steps = n_steps * warmup_proportion
    scheduler = WarmupLinearSchedule(
        optimizer, warmup_steps=warmup_steps, t_total=n_steps)
    return optimizer, scheduler

In [77]:
from pytorch_transformers import BertForNextSentencePrediction
from torch.utils.data import DataLoader

model = BertForNextSentencePrediction.from_pretrained(args.bert_model)
data_loader = DataLoader(dataset, args.batch_size, shuffle=True)

num_train_optimization_steps = len(data_loader) // args.n_epochs
optimizer, scheduler = init_optimizer(model, num_train_optimization_steps, args.learning_rate)

/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:72: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [89]:
from tqdm.auto import tqdm

tr_loss = 0
global_step = 0

for _ in tqdm(range(args.n_epochs), desc="Epochs"):
    for step, batch in tqdm(enumerate(data_loader), desc="Batches"):
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'token_type_ids': batch[2],
            'next_sentence_label': batch[3]}
        # forward propagation
        outputs = model(**inputs)
        loss = outputs[0].mean()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
        tr_loss += loss.item()
        
        if (step + 1) % args.gradient_accumulation_steps == 0:
            scheduler.step()
            optimizer.step()
            model.zero_grad()
            global_step += 1
        break

    break

/home/arthur/miniconda3/envs/bert/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:72: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [79]:
outputs = model(**inputs)

In [83]:
loss

tensor(11.8139, grad_fn=<NllLossBackward>)

In [80]:
loss = outputs[0].mean()

In [70]:
tmp_eval_loss, logits = outputs[:2]

In [39]:
eval_dataloader

In [50]:
eval_loss = 0.0
nb_eval_steps = 0
preds = None
out_label_ids = None
for batch in tqdm(eval_dataloader):
    if 0 in batch[3]:
        breake

In [49]:
1 in batch[3]

True